# 🚨 3-HOUR MC & TD CRASH COURSE — EXAM FOCUSED 🚨

> **You have 3 hours. No lectures attended. This is your survival guide.**

---

## ⏰ YOUR 3-HOUR BATTLE PLAN

| Time | What to Do |
|------|------------|
| **Hour 1** | Read concepts + Copy & Run MC code |
| **Hour 2** | Read TD section + Practice TD code |
| **Hour 3** | Full exam simulation |

---

## 🔹 Model-Free RL (30 sec)

- **Model-Based** (DP): You KNOW the environment
- **Model-Free** (MC & TD): You DON'T know. You LEARN by trying!

---

## 🔹 Monte Carlo (MC)

> **MC = Learn from COMPLETE episodes**

### Return Formula:
$$G_t = R_{t+1} + \gamma \cdot G_{t+1}$$

**Calculate G BACKWARDS!**

---

## 🔹 First-Visit vs Every-Visit

| Method | What to do |
|--------|------------|
| **First-Visit** | Use `visited` set, update first occurrence only |
| **Every-Visit** | NO `visited` set, update every occurrence |

---

## 🔹 Temporal Difference (TD)

> **TD = Learn DURING the journey, not after**

### TD Update Formula:
$$V(S_t) \leftarrow V(S_t) + \alpha \left[ R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \right]$$

| MC | TD |
|----|----|  
| Update AFTER episode | Update EVERY step |
| Uses actual G | Uses estimated return |

---
# 💻 PART 1: SETUP & CONSTANTS

In [20]:
import random
import matplotlib.pyplot as plt

# Constants
GRID_SIZE = 5
REWARD_GOAL = 10
REWARD_STEP = -1

---
# 📝 Task 1: Generate Episode

In [21]:
def generate_episode():
    """
    Generate an episode with random start and goal at corners.
    Returns: list of (state, reward) tuples
    """
    # Random start at corners
    agent_x = random.choice([0, GRID_SIZE-1])
    agent_y = random.choice([0, GRID_SIZE-1])

    # Random goal at corners
    goal_x = random.choice([0, GRID_SIZE-1])
    goal_y = random.choice([0, GRID_SIZE-1])

    episode = []
    steps = 0

    while (agent_x, agent_y) != (goal_x, goal_y) and steps < 20:
        state = (agent_x, agent_y)  # MUST be tuple!

        # Random action: 0=left, 1=up, 2=right, 3=down
        action = random.choice([0, 1, 2, 3])

        # Move (stay in grid boundaries!)
        if action == 0:    # left
            agent_x = max(agent_x - 1, 0)
        elif action == 1:  # up
            agent_y = max(agent_y - 1, 0)
        elif action == 2:  # right
            agent_x = min(agent_x + 1, GRID_SIZE - 1)
        elif action == 3:  # down
            agent_y = min(agent_y + 1, GRID_SIZE - 1)

        # Reward
        if (agent_x, agent_y) == (goal_x, goal_y):
            reward = REWARD_GOAL
        else:
            reward = REWARD_STEP

        episode.append((state, reward))
        steps += 1

    return episode

# TEST:
ep = generate_episode()
print(f"Episode: {ep}")
print(f"Length: {len(ep)}")

Episode: [((4, 0), -1), ((3, 0), -1), ((3, 0), -1), ((3, 0), -1), ((3, 0), -1), ((2, 0), -1), ((3, 0), -1), ((2, 0), -1), ((2, 0), -1), ((1, 0), -1), ((2, 0), -1), ((2, 0), -1), ((3, 0), -1), ((3, 1), -1), ((2, 1), -1), ((2, 2), -1), ((2, 1), -1), ((2, 2), -1), ((1, 2), -1), ((1, 1), -1)]
Length: 20


---
# 📝 Task 2: First-Visit MC Update

In [28]:
def first_visit_mc(episode, V, gamma=0.9):
    """
    First-visit Monte Carlo update.

    Args:
        episode: list of (state, reward)
        V: dict mapping state -> value
        gamma: discount factor
    """
    # Extract states and rewards
    states = [s for (s, r) in episode]
    rewards = [r for (s, r) in episode]

    G = 0.0
    visited = set()  # Track first visits!

    # Go BACKWARDS through episode
    for t in reversed(range(len(episode))):
        state = states[t]
        reward = rewards[t]

        # Calculate return
        G = gamma * G + reward

        # First-visit check
        if state not in visited:
            visited.add(state)

            # Update value
            old_value = V.get(state, 0.0)
            V[state] = old_value + G

    return V
  # RUN First-Visit MC:
V = {}
for _ in range(1000):
    episode = generate_episode()
    V = first_visit_mc(episode, V)

print(f"Learned {len(V)} states")
print(V)

Learned 25 states
{(3, 0): -915.4273991850314, (4, 0): -1190.4530215842558, (4, 1): -869.1483171781962, (3, 1): -1113.9355449857514, (4, 2): -803.2210257882847, (2, 4): -851.7449720015401, (3, 4): -805.4751014570975, (1, 4): -961.0845299017128, (0, 4): -1262.2949418132814, (0, 3): -893.1919640006225, (0, 2): -905.768727563359, (1, 2): -1072.8466951198761, (0, 1): -1031.9882734826126, (2, 2): -944.4376927091021, (2, 1): -1104.9348260246638, (2, 0): -922.3648735075295, (1, 1): -1176.7985242535701, (0, 0): -1450.95971148657, (3, 2): -1009.3774852316652, (3, 3): -1052.508613656553, (4, 3): -890.360000114809, (1, 0): -1097.8603513095545, (1, 3): -1024.622574288318, (2, 3): -1111.0170212925282, (4, 4): -1142.6692508717683}


---
# 📝 Task 3: Every-Visit MC Update

**ONLY DIFFERENCE: No `visited` set!**

In [24]:
def every_visit_mc(episode, V, gamma=0.9):
    """
    Every-visit Monte Carlo update.
    ONLY DIFFERENCE: No 'visited' set!
    """
    states = [s for (s, r) in episode]
    rewards = [r for (s, r) in episode]

    G = 0.0
    # NO visited set here!

    for t in reversed(range(len(episode))):
        state = states[t]
        reward = rewards[t]

        G = gamma * G + reward

        # Update EVERY time (no first-visit check)
        old_value = V.get(state, 0.0)
        V[state] = old_value + G

    return V

In [25]:
# RUN Every-Visit MC:
V2 = {}
for _ in range(1000):
    episode = generate_episode()
    V2 = every_visit_mc(episode, V2)

print(f"Every-visit learned {len(V2)} states")
print(V2)

Every-visit learned 25 states
{(4, 0): -5255.50089938093, (3, 0): -3114.4808273069757, (3, 1): -2446.818135748256, (2, 1): -2569.181179169013, (3, 2): -2354.5685570368505, (4, 1): -3127.8520147930158, (1, 1): -2884.4997832420995, (0, 1): -3322.869151701406, (0, 0): -6385.123414392148, (3, 3): -2296.8130032230856, (3, 4): -2537.4294759057825, (4, 4): -5319.054021994446, (4, 3): -3067.215912752964, (4, 2): -2776.817307994262, (1, 4): -2967.248151001546, (2, 4): -1900.7062327873834, (2, 3): -1985.6616169236022, (1, 3): -2491.7295788956358, (0, 3): -3194.8875975090223, (0, 2): -2465.513648754507, (1, 2): -2411.831999791466, (2, 2): -2281.240469602955, (0, 4): -5850.140370460864, (1, 0): -3216.649870807307, (2, 0): -2606.2171452983116}


---
# 📝 Task 4: TD(0) Update

**Key Formula:** `V[s] = V[s] + alpha × (reward + gamma × V[s'] - V[s])`

In [26]:
def td_update(V, state, reward, next_state, alpha=0.5, gamma=0.9, terminal=False):
    """
    TD(0) update for a single step.
    """
    old_value = V.get(state, 0.0)

    if terminal:
        next_value = 0
    else:
        next_value = V.get(next_state, 0.0)

    # TD update
    td_error = reward + gamma * next_value - old_value
    V[state] = old_value + alpha * td_error

    return V


def run_td_episode(V, alpha=0.5, gamma=0.9):
    """Run one episode with TD updates."""
    agent_x = random.choice([0, GRID_SIZE-1])
    agent_y = random.choice([0, GRID_SIZE-1])
    goal_x = random.choice([0, GRID_SIZE-1])
    goal_y = random.choice([0, GRID_SIZE-1])

    steps = 0

    while (agent_x, agent_y) != (goal_x, goal_y) and steps < 20:
        state = (agent_x, agent_y)

        action = random.choice([0, 1, 2, 3])

        if action == 0:
            agent_x = max(agent_x - 1, 0)
        elif action == 1:
            agent_y = max(agent_y - 1, 0)
        elif action == 2:
            agent_x = min(agent_x + 1, GRID_SIZE - 1)
        elif action == 3:
            agent_y = min(agent_y + 1, GRID_SIZE - 1)

        next_state = (agent_x, agent_y)

        if next_state == (goal_x, goal_y):
            reward = REWARD_GOAL
            terminal = True
        else:
            reward = REWARD_STEP
            terminal = False

        # TD UPDATE HAPPENS HERE (during episode!)
        V = td_update(V, state, reward, next_state, alpha, gamma, terminal)

        steps += 1

    return V
# RUN TD:
V_td = {}
for _ in range(1000):
    V_td = run_td_episode(V_td)

print(f"TD learned {len(V_td)} states")
print(V_td)

TD learned 25 states
{(0, 4): -9.542105967395653, (1, 4): 0.289157803461352, (1, 3): -9.470822127613678, (1, 2): -9.457766609015643, (2, 2): -9.339372955084993, (1, 1): -9.573580999335814, (2, 1): -8.50523568197342, (4, 0): -4.549163341699257, (3, 0): -9.22103575864789, (3, 1): -8.19510648293571, (3, 2): -8.601840518577443, (2, 0): -9.51017563623643, (4, 1): -3.9366260834089326, (2, 3): -9.474048482229826, (4, 2): -9.02236038036385, (4, 3): -9.422822045083143, (4, 4): -9.41479623805094, (3, 3): -9.436408202236734, (3, 4): -9.397681778392851, (2, 4): -9.367639220374532, (0, 0): -9.644435942612116, (1, 0): -9.580124600057573, (0, 1): -9.655005831410275, (0, 3): -9.593441302715934, (0, 2): -9.661727716602858}


---
# ❌ COMMON MISTAKES

### 1. Wrong Loop Direction
```python
# ❌ WRONG:
for t in range(len(episode)):

# ✅ RIGHT:
for t in reversed(range(len(episode))):
```

### 2. First-Visit Check
```python
# ❌ WRONG:
if state not in V:

# ✅ RIGHT:
if state not in visited:
    visited.add(state)
```

### 3. Boundaries
```python
# ❌ WRONG:
agent_x = min(agent_x + 1, GRID_SIZE)

# ✅ RIGHT:
agent_x = min(agent_x + 1, GRID_SIZE - 1)
```

---

# 🎯 FORMULAS RECAP

**MC Return:** `G = reward + gamma × (next G)` — GO BACKWARDS!

**TD Update:** `V[s] = V[s] + alpha × (reward + gamma × V[s'] - V[s])` — UPDATE EVERY STEP!

**First-Visit:** uses `visited` set | **Every-Visit:** no `visited` set